In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import lxml.html as lh
import re



In [2]:
url = 'https://wallethub.com/edu/happiest-states/6959'

In [3]:
response = requests.get(url)
soup = bs(response.text, 'html.parser')
print(soup.prettify())

<!-- PC [U] 2022-04-13T14:15:51.563Z -->
<!DOCTYPE html>
<html lang="en" scroll-region="active">
 <head>
  <link href="https://cdn.wallethub.com" rel="preconnect"/>
  <link href="https://www.googletagmanager.com" rel="preconnect"/>
  <link href="https://www.google-analytics.com" rel="preconnect"/>
  <link href="https://hello.myfonts.net" rel="preconnect"/>
  <link href="https://cdn.wallethub.com" rel="dns-prefetch"/>
  <link href="https://www.googletagmanager.com" rel="dns-prefetch"/>
  <link href="https://www.google-analytics.com" rel="dns-prefetch"/>
  <link href="https://hello.myfonts.net" rel="dns-prefetch"/>
  <link href="https://d2e70e9yced57e.cloudfront.net/common/wallethub/favicon2.ico" rel="shortcut icon"/>
  <link as="font" crossorigin="anonymous" href="https://cdn.wallethub.com/css/wh2015/framework/webfonts/proxima-nova/2D9E95_2_0_V60fad39_.woff2" rel="preload" type="font/woff2"/>
  <link as="image" href="https://cdn.wallethub.com/wallethub/posts/98271/happiest-states-in-ame

In [4]:
results= soup.find('div', class_='cardhub-edu-table-div')
print(results)

<div class="cardhub-edu-table-div sortable-main-1"><table class="cardhub-edu-table center-aligned sortable" style="width: 680px">
<thead>
<tr>
<th class="rank-numeric"><b>Overall Rank*</b></th>
<th><b>State</b></th>
<th class="rank-numeric"><b>Total Score</b></th>
<th class="rank-numeric"><b>Emotional &amp; Physical Well-Being</b></th>
<th class="rank-numeric"><b>Work Environment</b></th>
<th class="rank-numeric"><b>Community &amp; Environment</b></th>
</tr>
</thead>
<tbody><tr><td>1</td><td>Utah</td><td>72.94</td><td>9</td><td>1</td><td>1</td></tr><tr><td>2</td><td>Minnesota</td><td>67.52</td><td>2</td><td>4</td><td>9</td></tr><tr><td>3</td><td>Hawaii</td><td>66.16</td><td>8</td><td>15</td><td>3</td></tr><tr><td>4</td><td>California</td><td>63.82</td><td>4</td><td>20</td><td>11</td></tr><tr><td>5</td><td>North Dakota</td><td>63.25</td><td>13</td><td>5</td><td>6</td></tr><tr><td>6</td><td>South Dakota</td><td>63.08</td><td>14</td><td>6</td><td>4</td></tr><tr><td>7</td><td>Idaho</td><td

In [24]:
# the head will form our column names
body = results.find_all("tr")
# Head values (Column names) are the first items of the body list
head = body[0] # 0th item is the header row
body_rows = body[1:] # All other items becomes the rest of the rows

# Lets now iterate through the head HTML code and make list of clean headings

# Declare empty list to keep Columns names
headings = []
for item in head.find_all("th"): # loop through all th elements
    # convert the th elements to text and strip "\n"
    item = (item.text).rstrip("\n")
    # append the clean column name to headings
    headings.append(item)
print(headings)

['Overall Rank*', 'State', 'Total Score', 'Emotional & Physical Well-Being', 'Work Environment', 'Community & Environment']


In [25]:
all_rows = [] # will be a list for list for all rows
for row_num in range(len(body_rows)): # A row at a time
    row = [] # this will old entries for one row
    for row_item in body_rows[row_num].find_all("td"): #loop through all row entries
        # row_item.text removes the tags from the entries
        # the following regex is to remove \xa0 and \n and comma from row_item.text
        # xa0 encodes the flag, \n is the newline and comma separates thousands in numbers
        aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
        #append aa to row - note one row entry is being appended
        row.append(aa)
    # append one row to all_rows
    all_rows.append(row)

In [26]:
df = pd.DataFrame(data=all_rows,columns=headings)
df.head()

,Overall Rank*,State,Total Score,Emotional & Physical Well-Being,Work Environment,Community & Environment
0,1,Utah,72.94,9,1,1
1,2,Minnesota,67.52,2,4,9
2,3,Hawaii,66.16,8,15,3
3,4,California,63.82,4,20,11
4,5,North Dakota,63.25,13,5,6


In [27]:
# Add a placeholder for DC with nan values
df.loc[len(df.index)] = ['51', 'District of Columbia', 'NaN', 'NaN','NaN', 'NaN' ] 

In [28]:
df.dtypes

Overall Rank*                      object
State                              object
Total Score                        object
Emotional & Physical Well-Being    object
Work Environment                   object
Community & Environment            object
dtype: object

In [29]:
cols = ["Overall Rank*", 'Total Score','Emotional & Physical Well-Being', 'Work Environment', 'Community & Environment']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce', axis=1)


In [30]:
df.dtypes

Overall Rank*                      float64
State                               object
Total Score                        float64
Emotional & Physical Well-Being    float64
Work Environment                   float64
Community & Environment            float64
dtype: object

In [31]:
df=df.sort_values('State')
df.head()

,Overall Rank*,State,Total Score,Emotional & Physical Well-Being,Work Environment,Community & Environment
42,43.0,Alabama,42.99,45.0,33.0,16.0
41,42.0,Alaska,44.24,36.0,41.0,48.0
26,27.0,Arizona,52.76,27.0,28.0,33.0
48,49.0,Arkansas,37.47,50.0,35.0,15.0
3,4.0,California,63.82,4.0,20.0,11.0


In [32]:
df.reset_index(inplace=True)

In [33]:
df= df.drop(columns="index")
df.head()

,Overall Rank*,State,Total Score,Emotional & Physical Well-Being,Work Environment,Community & Environment
0,43.0,Alabama,42.99,45.0,33.0,16.0
1,42.0,Alaska,44.24,36.0,41.0,48.0
2,27.0,Arizona,52.76,27.0,28.0,33.0
3,49.0,Arkansas,37.47,50.0,35.0,15.0
4,4.0,California,63.82,4.0,20.0,11.0


In [34]:
df=df.rename(columns={"Overall Rank*": "Happiness_Rank", "Total Score": "Happiness_Score", "Emotional & Physical Well-Being": "Emotional_&_Physical_Well-Being_Rank", "Work Environment": "Work_Environment_Rank", "Community & Environment": "Community_&_Environment_Rank"})

In [35]:
df['inv_hap_rank'] = df['Happiness_Rank'].rank(method='max', na_option='bottom', ascending= False).astype(int)

In [36]:
df

,Happiness_Rank,State,Happiness_Score,Emotional_&_Physical_Well-Being_Rank,Work_Environment_Rank,Community_&_Environment_Rank,inv_hap_rank
0,43.0,Alabama,42.99,45.0,33.0,16.0,9
1,42.0,Alaska,44.24,36.0,41.0,48.0,10
2,27.0,Arizona,52.76,27.0,28.0,33.0,25
3,49.0,Arkansas,37.47,50.0,35.0,15.0,3
4,4.0,California,63.82,4.0,20.0,11.0,48
5,22.0,Colorado,53.99,17.0,17.0,47.0,30
6,11.0,Connecticut,60.71,5.0,46.0,17.0,41
7,26.0,Delaware,52.77,23.0,36.0,36.0,26
8,51.0,District of Columbia,NaN,NaN,NaN,NaN,1
9,34.0,Florida,50.55,21.0,42.0,49.0,18


In [37]:
df.to_csv("../csv/happiness.csv", index= False)

In [38]:
df.to_json('../json/happiness.json', orient='records')